In [6]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image
import sklearn.metrics as mt
import os
from os import path
import copy
from importlib import reload
import itertools
import trainer as tr
import dataset as ds
import predict as pr
reload(tr)
reload(ds)
reload(pr)
# %matplotlib inline


<module 'predict' from '/Users/habbes/code/class/bd4h/chexpert/src/predict.py'>

In [7]:
os.chdir('..')

In [8]:
val = pd.read_csv(ds.VALID_CSV)
val.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
val['Path'] = val['Path'].apply(lambda p: path.join('./data', p))

In [11]:
val

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,./data/CheXpert-v1.0-small/valid/patient64541/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,./data/CheXpert-v1.0-small/valid/patient64542/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,./data/CheXpert-v1.0-small/valid/patient64542/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,./data/CheXpert-v1.0-small/valid/patient64543/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,./data/CheXpert-v1.0-small/valid/patient64544/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,./data/CheXpert-v1.0-small/valid/patient64545/...,Female,55,Frontal,AP,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,./data/CheXpert-v1.0-small/valid/patient64546/...,Male,56,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7,./data/CheXpert-v1.0-small/valid/patient64547/...,Male,59,Frontal,PA,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,./data/CheXpert-v1.0-small/valid/patient64547/...,Male,59,Frontal,PA,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,./data/CheXpert-v1.0-small/valid/patient64547/...,Male,59,Lateral,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
paths = pd.DataFrame(data=val['Path'])
paths.head()

,Path
0,./data/CheXpert-v1.0-small/valid/patient64541/...
1,./data/CheXpert-v1.0-small/valid/patient64542/...
2,./data/CheXpert-v1.0-small/valid/patient64542/...
3,./data/CheXpert-v1.0-small/valid/patient64543/...
4,./data/CheXpert-v1.0-small/valid/patient64544/...


In [15]:
paths.to_csv('dummy_valid_paths.csv', index=False)